# Guidance for training a model with your own data

## 1. Import the necessary packages

In [2]:
import argparse
import os
import random

import numpy as np
import pandas as pd
import torch

from softs.exp.exp_custom import Exp_Custom

## 2. Define the hyperparameters

In [3]:
# fix seed for reproducibility
fix_seed = 7
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)
torch.set_num_threads(6)

# basic config
config = {
    # dataset settings
    "root_path": "./dataset/ETT-small/",
    "data_path": "ETTm1.csv",
    "data": "ETTm1",
    "features": "MS",
    "freq": "B",
    "seq_len": 300,
    "pred_len": 20,
    # model settings
    "model": "SOFTS",
    "checkpoints": "./checkpoints/",
    "d_model": 1024,
    "d_core": 1024,
    "d_ff": 1024,
    "e_layers": 16,
    "learning_rate": 0.0001,
    "lradj": "cosine",
    "train_epochs": 50,
    "patience": 3,
    "batch_size": 256,
    "dropout": 0.0,
    "activation": "gelu",
    "use_norm": True,
    "loss_func": "huber",
    # system settings
    "num_workers": 0,
    "use_gpu": True,
    "gpu": "0",
    "save_model": True,
    "predict_all": True,
    "mixed_precision": True,
    "optimizer": "AdamW",  # SGD / Adam / AdamW
}

parser = argparse.ArgumentParser(description='SOFTS')
args = parser.parse_args([])
args.__dict__.update(config)
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

print('Args in experiment:')
print(args)

Args in experiment:
Namespace(root_path='./dataset/ETT-small/', data_path='ETTm1.csv', data='ETTm1', features='MS', freq='B', seq_len=300, pred_len=20, model='SOFTS', checkpoints='./checkpoints/', d_model=512, d_core=512, d_ff=512, e_layers=16, learning_rate=0.0001, lradj='cosine', train_epochs=50, patience=3, batch_size=32, dropout=0.0, activation='gelu', use_norm=True, loss_func='huber', num_workers=0, use_gpu=False, gpu='0', save_model=True, predict_all=True, mixed_precision=True, optimizer='AdamW')


In [3]:
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()  # take environment variables from .env.

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

alchemyEngine = create_engine(
    db_url,
    pool_recycle=3600,
)

In [4]:
# query = """
# with cte as (
# SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
# FROM index_daily_em_view 
# where symbol = '399673' 
# order by date desc
# limit 1200
# ) select * from cte order by date
# """
query = """
with cte as (
SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
FROM index_daily_em_view 
where symbol = '399673'
) select * from cte order by date
"""

df = pd.read_sql(query, alchemyEngine, parse_dates=["date"])

In [5]:
df

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
0,2014-06-18,1344.05,1328.84,1351.16,1328.81,3938637.0,8.436718e+09,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-06-19,1327.54,1281.12,1335.44,1269.53,4532795.0,9.384841e+09,-0.09783,0.49667,-4.46329,15.08537,11.23805,-3.59110
2,2014-06-20,1281.14,1294.98,1297.99,1273.18,3111206.0,6.570250e+09,0.00156,1.31682,-0.61977,-31.36231,-29.99082,1.08187
3,2014-06-23,1295.94,1324.14,1330.24,1295.94,3615497.0,7.837101e+09,0.07413,2.72282,0.07413,16.20886,19.28164,2.25177
4,2014-06-24,1322.55,1331.38,1335.14,1320.99,3265263.0,6.941363e+09,-0.12008,0.83073,-0.23789,-9.68702,-11.42946,0.54677
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,2024-07-04,1606.52,1588.34,1616.41,1588.34,9225758.0,2.946956e+10,0.46401,1.08248,-0.67288,-9.95871,-4.00640,-0.67288
2445,2024-07-05,1590.68,1593.03,1600.13,1572.45,9580758.0,3.054292e+10,0.14732,0.74228,-1.00042,3.84792,3.64226,0.29528
2446,2024-07-08,1588.87,1566.89,1590.42,1562.82,9333888.0,2.947581e+10,-0.26114,-0.16384,-1.89639,-2.57673,-3.49381,-1.64090
2447,2024-07-09,1564.27,1596.29,1604.00,1556.98,13485580.0,4.241111e+10,-0.16721,2.36839,-0.63246,44.47977,43.88446,1.87633


In [5]:
df = df.dropna()
df

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
1,2014-06-19,1327.54,1281.12,1335.44,1269.53,4532795.0,9.384841e+09,-0.09783,0.49667,-4.46329,15.08537,11.23805,-3.59110
2,2014-06-20,1281.14,1294.98,1297.99,1273.18,3111206.0,6.570250e+09,0.00156,1.31682,-0.61977,-31.36231,-29.99082,1.08187
3,2014-06-23,1295.94,1324.14,1330.24,1295.94,3615497.0,7.837101e+09,0.07413,2.72282,0.07413,16.20886,19.28164,2.25177
4,2014-06-24,1322.55,1331.38,1335.14,1320.99,3265263.0,6.941363e+09,-0.12008,0.83073,-0.23789,-9.68702,-11.42946,0.54677
5,2014-06-25,1330.12,1322.78,1330.70,1310.50,3214760.0,6.779457e+09,-0.09464,-0.05107,-1.56830,-1.54667,-2.33248,-0.64595
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,2024-07-04,1606.52,1588.34,1616.41,1588.34,9225758.0,2.946956e+10,0.46401,1.08248,-0.67288,-9.95871,-4.00640,-0.67288
2445,2024-07-05,1590.68,1593.03,1600.13,1572.45,9580758.0,3.054292e+10,0.14732,0.74228,-1.00042,3.84792,3.64226,0.29528
2446,2024-07-08,1588.87,1566.89,1590.42,1562.82,9333888.0,2.947581e+10,-0.26114,-0.16384,-1.89639,-2.57673,-3.49381,-1.64090
2447,2024-07-09,1564.27,1596.29,1604.00,1556.98,13485580.0,4.241111e+10,-0.16721,2.36839,-0.63246,44.47977,43.88446,1.87633


In [7]:
df = df[["date", "change_rate"]]
df

,date,change_rate
1,2014-06-19,-3.59110
2,2014-06-20,1.08187
3,2014-06-23,2.25177
4,2014-06-24,0.54677
5,2014-06-25,-0.64595
...,...,...
2441,2024-07-01,-0.18617
2442,2024-07-02,-0.84210
2443,2024-07-03,-0.07124
2444,2024-07-04,-0.67288


In [7]:
df.describe()

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
count,2445,2445.000000,2445.000000,2445.000000,2445.000000,2.445000e+03,2.445000e+03,2445.000000,2445.000000,2445.000000,2445.000000,2445.000000,2445.000000
mean,2019-06-23 11:24:57.423312896,2020.078687,2020.881059,2045.388863,1995.243493,1.078333e+07,3.282570e+10,-0.040179,1.189170,-1.198846,2.279401,2.162237,0.027744
min,2014-06-19 00:00:00,960.230000,970.750000,993.160000,959.500000,2.167637e+06,4.935305e+09,-7.981020,-4.760830,-9.709820,-71.871310,-73.844110,-9.675610
25%,2016-12-16 00:00:00,1547.370000,1548.460000,1563.760000,1529.890000,7.733564e+06,1.517756e+10,-0.312020,0.294770,-1.713120,-12.293890,-12.005060,-1.026860
50%,2019-06-24 00:00:00,1902.310000,1902.080000,1925.580000,1881.900000,1.006484e+07,3.030508e+10,-0.036700,0.865930,-0.894380,-1.115390,-1.600920,-0.032460
75%,2021-12-24 00:00:00,2437.500000,2445.450000,2481.210000,2398.780000,1.298826e+07,4.628830e+10,0.299480,1.757850,-0.333100,12.133400,12.575960,1.104090
max,2024-07-05 00:00:00,3895.610000,3871.400000,3929.050000,3720.170000,3.395136e+07,1.035382e+11,8.111950,8.111950,3.007580,306.093110,297.629870,7.464270
std,NaN,633.343884,632.534140,643.078920,620.742851,4.550688e+06,1.968276e+10,0.851786,1.396648,1.475941,23.116774,22.244160,2.012874


## 3. Prepare the dataset
Organize your data in the following format:
- The dataset should be a csv file.
- If there is a time feature, the first column contains timestamps in the format 'YYYY-MM-DD HH:MM:SS'. If there's no time feature, the dataset starts directly with the features.
- If the parameter `features` is 'M', the following columns are both the features and the targets. If `features` is 'MS', the following columns are the features, and the last column is the target.

In [6]:
# load data
# data = pd.read_csv(os.path.join(args.root_path, args.data_path))
# print(data.head())

# split data
end = int(len(df) * 0.9)
train_data = df.iloc[: end]
vali_data = df.iloc[end - args.seq_len: ]
# test_data = df.iloc[1100 - args.seq_len: ]

# optional: scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
if 'date' in train_data.columns:
    scaler.fit(train_data.iloc[:, 1:])
    train_data.iloc[:, 1:] = scaler.transform(train_data.iloc[:, 1:])
    vali_data.iloc[:, 1:] = scaler.transform(vali_data.iloc[:, 1:])
    # test_data.iloc[:, 1:] = scaler.transform(test_data.iloc[:, 1:])
else:
    scaler.fit(train_data.iloc[:, :])
    train_data.iloc[:, :] = scaler.transform(train_data.iloc[:, :])
    vali_data.iloc[:, :] = scaler.transform(vali_data.iloc[:, :])
    # test_data.iloc[:, :] = scaler.transform(test_data.iloc[:, :])

In [8]:
train_data

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
1,2014-06-19,-1.081418,-1.154688,-1.092350,-1.153046,-1.336502,-1.114820,-0.069108,-0.511844,-2.129088,0.554889,0.403232,-1.763756
2,2014-06-20,-1.151601,-1.133696,-1.148141,-1.147414,-1.638626,-1.253585,0.045639,0.069440,0.396457,-1.457571,-1.436750,0.504710
3,2014-06-23,-1.129215,-1.089531,-1.100097,-1.112292,-1.531451,-1.191127,0.129422,1.065946,0.852413,0.603567,0.762205,1.072631
4,2014-06-24,-1.088965,-1.078566,-1.092797,-1.073637,-1.605885,-1.235288,-0.094796,-0.275078,0.647388,-0.518436,-0.608384,0.244949
5,2014-06-25,-1.077515,-1.091591,-1.099412,-1.089824,-1.616618,-1.243270,-0.065426,-0.900056,-0.226814,-0.165735,-0.202400,-0.334050
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,2023-06-30,-3.089326,-3.094850,-3.081672,-3.111948,-2.299837,-1.577509,0.056768,-0.325445,1.132760,-0.081955,-0.065062,0.363485
2200,2023-07-03,-3.089225,-3.094816,-3.081640,-3.111837,-2.299837,-1.577509,0.649831,-0.835426,1.370272,-0.105093,-0.101897,0.130417
2201,2023-07-04,-3.089225,-3.094802,-3.081648,-3.111836,-2.299837,-1.577509,-0.263729,-1.260951,1.087027,-0.081574,-0.120524,0.037318
2202,2023-07-05,-3.089190,-3.094840,-3.081624,-3.111839,-2.299837,-1.577509,0.315951,-1.157918,0.942211,-0.140590,-0.123574,-0.211376


In [9]:
vali_data

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
2144,2023-04-07,-3.088777,-3.094320,-3.081164,-3.111356,-2.299837,-1.577509,-0.186314,-0.783299,1.229563,-0.121971,-0.123627,0.184005
2145,2023-04-10,-3.088683,-3.094293,-3.081143,-3.111308,-2.299837,-1.577509,0.952360,-1.044104,1.215068,-0.093396,-0.057758,0.091705
2146,2023-04-11,-3.088674,-3.094307,-3.081107,-3.111307,-2.299837,-1.577509,0.472155,-0.959648,0.996899,-0.125721,-0.122584,-0.092311
2147,2023-04-12,-3.088705,-3.094343,-3.081173,-3.111341,-2.299837,-1.577509,0.074096,-1.459296,0.846250,-0.097216,-0.121386,-0.188209
2148,2023-04-13,-3.088770,-3.094397,-3.081219,-3.111374,-2.299837,-1.577509,-0.653830,-1.564263,0.880580,-0.113214,-0.106981,-0.269051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,2024-07-04,-0.659439,-0.689383,-0.673776,-0.661082,-0.339128,-0.124610,0.579544,-0.096650,0.361559,-0.530207,-0.277104,-0.347123
2445,2024-07-05,-0.683398,-0.682280,-0.698029,-0.685603,-0.263682,-0.071691,0.213921,-0.337767,0.146335,0.067999,0.064244,0.122864
2446,2024-07-08,-0.686136,-0.721870,-0.712495,-0.700463,-0.316148,-0.124302,-0.257652,-0.979982,-0.442399,-0.210365,-0.254228,-0.817043
2447,2024-07-09,-0.723345,-0.677342,-0.692264,-0.709475,0.566192,0.513430,-0.149209,0.814743,0.388119,1.828474,1.860192,0.890375


In [5]:
Exp = Exp_Custom(args)
print(f"Estimated memory needed: {Exp.memory_demand() / (1024 ** 2):.2f} MB")

Estimated memory needed: 356.38 MB


## 4. Train and Evaluate the model


In [11]:
Exp = Exp_Custom(args)
setting = f'{args.data}_{args.model}_{args.seq_len}_{args.pred_len}'
print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
Exp.train(setting=setting, train_data=train_data, vali_data=vali_data, test_data=None)
print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
Exp.test(setting=setting, test_data=vali_data)

Use CPU


>>>>>>>start training : ETTm1_SOFTS_60_20>>>>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>testing : ETTm1_SOFTS_60_20<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


(0.7318887484402247, 0.5751916130559634, 0.24959460243714593)

## 5. Get predictions by the model

In [24]:
vali_data

,date,change_rate
2140,2023-03-31,0.035238
2141,2023-04-03,0.351201
2142,2023-04-04,-0.334935
2143,2023-04-06,-0.049566
2144,2023-04-07,0.183379
...,...,...
2440,2024-06-28,-0.643194
2441,2024-07-01,-0.110980
2442,2024-07-02,-0.429146
2443,2024-07-03,-0.055232


In [41]:
# get predictions
predictions = Exp.predict(setting=setting, pred_data=vali_data)
print(predictions.shape)

(246, 20, 12)


In [42]:
last_pred2 = predictions[-1, :, :]

In [43]:
it_padded_pred = scaler.inverse_transform(last_pred2)

In [44]:
it_padded_pred

array([[ 1.72926062e+03,  1.72686865e+03,  1.74489355e+03,
         1.71462952e+03,  1.05938800e+07,  3.98001193e+10,
        -5.89669421e-02,  8.62988114e-01, -9.07311678e-01,
         2.63056707e+00,  2.58245730e+00, -1.86177105e-01],
       [ 1.72287305e+03,  1.72105542e+03,  1.73923853e+03,
         1.70885339e+03,  1.04333480e+07,  3.88074783e+10,
        -1.32070065e-01,  8.33381534e-01, -9.16531861e-01,
         1.28995299e+00,  4.22393382e-01, -2.49974847e-01],
       [ 1.72874341e+03,  1.72724268e+03,  1.74481958e+03,
         1.71453772e+03,  1.05205700e+07,  3.92841380e+10,
        -1.84778079e-01,  7.83455849e-01, -9.80892599e-01,
         1.20063233e+00,  5.72999656e-01, -2.28112161e-01],
       [ 1.72846277e+03,  1.72585388e+03,  1.74338196e+03,
         1.71407666e+03,  1.04830710e+07,  3.90761841e+10,
        -1.04344524e-01,  7.14486778e-01, -9.46922243e-01,
         1.34852231e-01, -5.79224765e-01, -2.79890388e-01],
       [ 1.72550977e+03,  1.72416235e+03,  1.7412616

In [29]:
it_padded_pred.shape

(20, 1)

In [45]:
it_padded_pred[:, -1]

array([-0.1861771 , -0.24997485, -0.22811216, -0.2798904 , -0.15068327,
       -0.2187123 , -0.3109654 , -0.22825038,  0.13667627, -0.45120478,
       -0.06703919, -0.27190477, -0.16351332, -0.3224053 , -0.25958896,
       -0.03928372, -0.23652971, -0.26447386, -0.20533572,  0.1468694 ],
      dtype=float32)

In [46]:
it_padded_pred[:, -1].mean()

-0.19252497

In [47]:
it_padded_pred[:, -1].sum()

-3.8504994

In [32]:
train_data.shape

(2199, 2)

In [23]:
# Assuming last_pred is the single column prediction with shape (20, 1)
# and train_data has 13 columns

# Determine the number of columns the scaler was fitted on
num_columns = (
    train_data.shape[1] - 1 if "date" in train_data.columns else train_data.shape[1]
)

# Create an array of zeros with the same number of rows and columns as the scaler's fitted data
padded_pred = np.zeros((last_pred.shape[0], num_columns))

print(padded_pred.shape)

# Insert the single column prediction into the corresponding column (e.g., the first column)
padded_pred[:, 0] = last_pred[:, 0]

# Apply inverse_transform
it_padded_pred = scaler.inverse_transform(padded_pred)

# Extract the inverse transformed single column
it_last_pred = it_padded_pred[:, 0]

print(it_last_pred)

(20, 12)
[3885.40793131 3696.25140262 3774.6123283  3731.39867308 3809.44623464
 3699.65109014 3648.76137504 3721.66227388 3780.73833635 3730.91085114
 3769.92774081 3719.35071022 3732.21961064 3754.28062473 3555.72240312
 3795.2652735  3655.648892   3658.06218308 3602.10752726 3869.83301425]


In [13]:
Exp.metrics

{'epoch': 5,
 'MAE_val': 0.8422023778737977,
 'RMSE_val': 1.1137292115524855,
 'Loss_val': 0.5000086828719738,
 'MAE': 0.3929128,
 'RMSE': 0.5931921,
 'Loss': 0.17228595167398453}